In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import math
import glob
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

2025-02-14 10:40:04.462742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [3]:
'''
        Each file contains nine columns and a different number of rows depending on the test length.
        
        1st column is the acceleration data in the X axis measured by the sensor ADXL345.
        2nd column is the acceleration data in the Y axis measured by the sensor ADXL345.
        3rd column is the acceleration data in the Z axis measured by the sensor ADXL345.

        4th column is the rotation data in the X axis measured by the sensor ITG3200.
        5th column is the rotation data in the Y axis measured by the sensor ITG3200.
        6th column is the rotation data in the Z axis measured by the sensor ITG3200.

        7th column is the acceleration data in the X axis measured by the sensor MMA8451Q.
        8th column is the acceleration data in the Y axis measured by the sensor MMA8451Q.
        9th column is the acceleration data in the Z axis measured by the sensor MMA8451Q.
        
        Data are in bits with the following characteristics:

        ADXL345:
        Resolution: 13 bits
        Range: +-16g

        ITG3200
        Resolution: 16 bits
        Range: +-2000°/s

        MMA8451Q:
        Resolution: 14 bits
        Range: +-8g
'''

'\n        Each file contains nine columns and a different number of rows depending on the test length.\n        \n        1st column is the acceleration data in the X axis measured by the sensor ADXL345.\n        2nd column is the acceleration data in the Y axis measured by the sensor ADXL345.\n        3rd column is the acceleration data in the Z axis measured by the sensor ADXL345.\n\n        4th column is the rotation data in the X axis measured by the sensor ITG3200.\n        5th column is the rotation data in the Y axis measured by the sensor ITG3200.\n        6th column is the rotation data in the Z axis measured by the sensor ITG3200.\n\n        7th column is the acceleration data in the X axis measured by the sensor MMA8451Q.\n        8th column is the acceleration data in the Y axis measured by the sensor MMA8451Q.\n        9th column is the acceleration data in the Z axis measured by the sensor MMA8451Q.\n        \n        Data are in bits with the following characteristics:\

In [4]:
datapath = "./data/SisFall.csv"

df = pd.read_csv(datapath, index_col=0)
df.head(10)

,ADXL_x,ADXL_y,ADXL_z,ITG_x,ITG_y,ITG_z,MMA_x,MMA_y,MMA_z,subject,activity,trial
NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,SA01,D01,R01
0.0,17.0,-179.0,-99.0,-18.0,-504.0,-352.0,76.0,-697.0,-279.0,SA01,D01,R01
1.0,15.0,-174.0,-90.0,-53.0,-568.0,-306.0,48.0,-675.0,-254.0,SA01,D01,R01
2.0,1.0,-176.0,-81.0,-84.0,-613.0,-271.0,-2.0,-668.0,-221.0,SA01,D01,R01
3.0,-10.0,-180.0,-77.0,-104.0,-647.0,-227.0,-34.0,-697.0,-175.0,SA01,D01,R01
4.0,-21.0,-191.0,-63.0,-128.0,-675.0,-191.0,-74.0,-741.0,-133.0,SA01,D01,R01
5.0,-37.0,-225.0,-59.0,-146.0,-700.0,-159.0,-110.0,-840.0,-103.0,SA01,D01,R01
6.0,-36.0,-243.0,-46.0,-166.0,-722.0,-131.0,-147.0,-948.0,-75.0,SA01,D01,R01
7.0,-44.0,-271.0,-38.0,-190.0,-738.0,-107.0,-171.0,-1073.0,-38.0,SA01,D01,R01
8.0,-51.0,-312.0,-33.0,-210.0,-752.0,-90.0,-200.0,-1191.0,3.0,SA01,D01,R01


In [5]:
print (df.columns)
print (set(df['subject']))
features = df.copy()

# dnum = 200
# fnum = 365

Index(['ADXL_x', 'ADXL_y', 'ADXL_z', 'ITG_x', 'ITG_y', 'ITG_z', 'MMA_x',
       'MMA_y', 'MMA_z', 'subject', 'activity', 'trial'],
      dtype='object')
{'SE12', 'SA19', 'SA18', 'SA20', 'SA21', 'SE02', 'SA16', 'SA23', 'SE11', 'SE10', 'SA09', 'SE05', 'SA11', 'SA15', 'SE04', 'SE06', 'SE08', 'SA10', 'SA12', 'SA08', 'SE09', 'SA06', 'SE07', 'SA22', 'SA05', 'SA04', 'SA01', 'SA13', 'SA14', 'SE01', 'SA17', 'SE03', 'SA07', 'SA02', 'SE15', 'SE13', 'SA03', 'SE14'}


In [6]:
def drop_outlier(features) :
    sensor_columns = [col for col in features.columns if col not in ['activity', 'subject', 'trial']]
    
    z_scores = pd.DataFrame(index=features.index)
    for col in sensor_columns:
        z_scores[col] = (features[col] - features[col].mean()) / features[col].std()
    
    mask = (np.abs(z_scores) < 3).all(axis=1)
    filtered_features = features[mask].copy()
    
    print(f"original : {len(features)}")
    print(f"after : {len(filtered_features)}")
    print(f"number of deleted data : {len(features) - len(filtered_features)}")
    
    return filtered_features

features = drop_outlier(df)

original : 15838429
after : 14906814
number of deleted data : 931615


In [7]:
D01 = features[(features['activity']=='D01') & (features['subject']=='SA01')].copy()
D02 = features[(features['activity']=='D02') & (features['subject']=='SA01')].copy()
D03 = features[(features['activity']=='D03') & (features['subject']=='SA01')].copy()
D04 = features[(features['activity']=='D04') & (features['subject']=='SA01')].copy()
D05 = features[(features['activity']=='D05') & (features['subject']=='SA01')].copy()
D06 = features[(features['activity']=='D06') & (features['subject']=='SA01')].copy()
D07 = features[(features['activity']=='D07') & (features['subject']=='SA01')].copy()
D08 = features[(features['activity']=='D08') & (features['subject']=='SA01')].copy()
D09 = features[(features['activity']=='D09') & (features['subject']=='SA01')].copy()
D10 = features[(features['activity']=='D10') & (features['subject']=='SA01')].copy()
D11 = features[(features['activity']=='D11') & (features['subject']=='SA01')].copy()
D12 = features[(features['activity']=='D12') & (features['subject']=='SA01')].copy()
D13 = features[(features['activity']=='D13') & (features['subject']=='SA01')].copy()
D14 = features[(features['activity']=='D14') & (features['subject']=='SA01')].copy()
D15 = features[(features['activity']=='D15') & (features['subject']=='SA01')].copy()
D16 = features[(features['activity']=='D16') & (features['subject']=='SA01')].copy()
D17 = features[(features['activity']=='D17') & (features['subject']=='SA01')].copy()
D18 = features[(features['activity']=='D18') & (features['subject']=='SA01')].copy()
D19 = features[(features['activity']=='D19') & (features['subject']=='SA01')].copy()

In [8]:
F01 = features[(features['activity']=='F01') & (features['subject']=='SA01')].copy()
F02 = features[(features['activity']=='F02') & (features['subject']=='SA01')].copy()
F03 = features[(features['activity']=='F03') & (features['subject']=='SA01')].copy()
F04 = features[(features['activity']=='F04') & (features['subject']=='SA01')].copy()
F05 = features[(features['activity']=='F05') & (features['subject']=='SA01')].copy()
F06 = features[(features['activity']=='F06') & (features['subject']=='SA01')].copy()
F07 = features[(features['activity']=='F07') & (features['subject']=='SA01')].copy()
F08 = features[(features['activity']=='F08') & (features['subject']=='SA01')].copy()
F09 = features[(features['activity']=='F09') & (features['subject']=='SA01')].copy()
F10 = features[(features['activity']=='F10') & (features['subject']=='SA01')].copy()
F11 = features[(features['activity']=='F11') & (features['subject']=='SA01')].copy()
F12 = features[(features['activity']=='F12') & (features['subject']=='SA01')].copy()
F13 = features[(features['activity']=='F13') & (features['subject']=='SA01')].copy()
F14 = features[(features['activity']=='F14') & (features['subject']=='SA01')].copy()
F15 = features[(features['activity']=='F15') & (features['subject']=='SA01')].copy()

In [9]:
df = pd.DataFrame()
df = pd.concat([df, F01, F02, F03, F04, F05, F06, F07, F08, F09, F10, F11, F12, F13, F14, F15])
df = pd.concat([df, D01, D02, D03, D04, D05, D06, D07, D08, D09, D10, D11, D15, D16, D17, D18, D19])

In [10]:
df['activity'].value_counts()

activity
D05    24697
D17    24649
D06    23317
D01    19973
D02    18097
F14    14793
F11    14664
F15    14558
F12    14519
F09    14511
F13    14490
F08    14483
F10    14371
F07    14281
F01    14167
F03    14161
F02    14157
F04    13978
F06    13754
D03    13203
F05    12051
D07    12005
D15    12005
D16    12005
D09    11998
D19    11884
D08    11876
D11    11866
D10    11864
D18    11507
D04     7221
Name: count, dtype: int64

In [11]:
features = df.copy()

In [12]:
features = features.replace({'activity':'D01'},0)
features = features.replace({'activity':'D01'},0)
features = features.replace({'activity':'D02'},0)
features = features.replace({'activity':'D03'},0)
features = features.replace({'activity':'D04'},0)
features = features.replace({'activity':'D05'},0)
features = features.replace({'activity':'D06'},0)
features = features.replace({'activity':'D07'},0)
features = features.replace({'activity':'D08'},0)
features = features.replace({'activity':'D09'},0)
features = features.replace({'activity':'D10'},0)
features = features.replace({'activity':'D11'},0)
features = features.replace({'activity':'D12'},0)
features = features.replace({'activity':'D13'},0)
features = features.replace({'activity':'D14'},0)
features = features.replace({'activity':'D15'},0)
features = features.replace({'activity':'D16'},0)
features = features.replace({'activity':'D17'},0)
features = features.replace({'activity':'D18'},0)
features = features.replace({'activity':'D19'},0)
features = features.replace({'activity':'F01'},1)
features = features.replace({'activity':'F02'},1)
features = features.replace({'activity':'F03'},1)
features = features.replace({'activity':'F04'},1)
features = features.replace({'activity':'F05'},1)
features = features.replace({'activity':'F06'},1)
features = features.replace({'activity':'F07'},1)
features = features.replace({'activity':'F08'},1)
features = features.replace({'activity':'F09'},1)
features = features.replace({'activity':'F10'},1)
features = features.replace({'activity':'F11'},1)
features = features.replace({'activity':'F12'},1)
features = features.replace({'activity':'F13'},1)
features = features.replace({'activity':'F14'},1)
features = features.replace({'activity':'F15'},1)
features = features.replace({'activity':'F15'},1)

/var/folders/_z/gryfr07n59jgb3wrd062h1ym0000gn/T/ipykernel_34013/4220643108.py:35: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features = features.replace({'activity':'F15'},1)


In [13]:
features['activity'].value_counts()

activity
0    238167
1    212938
Name: count, dtype: int64

In [14]:
features

,ADXL_x,ADXL_y,ADXL_z,ITG_x,ITG_y,ITG_z,MMA_x,MMA_y,MMA_z,subject,activity,trial
NaN,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,SA01,1,R01
0.0,-9.0,-257.0,-25.0,84.0,247.0,27.0,-120.0,-987.0,63.0,SA01,1,R01
1.0,-3.0,-263.0,-23.0,99.0,258.0,35.0,-110.0,-1016.0,68.0,SA01,1,R01
2.0,-1.0,-270.0,-22.0,114.0,272.0,45.0,-94.0,-1037.0,69.0,SA01,1,R01
3.0,1.0,-277.0,-24.0,127.0,286.0,57.0,-81.0,-1062.0,69.0,SA01,1,R01
...,...,...,...,...,...,...,...,...,...,...,...,...
2395.0,5.0,-258.0,-67.0,-36.0,-5.0,-5.0,2.0,-1002.0,-144.0,SA01,0,R05
2396.0,1.0,-260.0,-67.0,-37.0,-5.0,-6.0,-5.0,-1000.0,-143.0,SA01,0,R05
2397.0,-2.0,-258.0,-67.0,-37.0,-5.0,-7.0,-8.0,-1003.0,-142.0,SA01,0,R05
2398.0,1.0,-258.0,-66.0,-38.0,-5.0,-6.0,4.0,-1002.0,-143.0,SA01,0,R05


In [15]:
print (set(features['subject']))

{'SA01'}


In [16]:
features = features[[ft for ft in features.columns if ft not in ['subject', 'trial']]]
print(features.columns)

Index(['ADXL_x', 'ADXL_y', 'ADXL_z', 'ITG_x', 'ITG_y', 'ITG_z', 'MMA_x',
       'MMA_y', 'MMA_z', 'activity'],
      dtype='object')


In [17]:
from sklearn.preprocessing import StandardScaler

feature_columns = [col for col in features.columns if col != 'activity']

scaler = StandardScaler()

features_scaled = features.copy()
features_scaled[feature_columns] = scaler.fit_transform(features[feature_columns])

In [18]:
adl_count = features_scaled['activity'].value_counts()[0]
fall_count = features_scaled['activity'].value_counts()[1]

adl_mask = features_scaled['activity'] == 0
adl_data = features_scaled[adl_mask]
adl1 = adl_data.iloc[:int(adl_count * 0.6)]
adl2 = adl_data.iloc[int(adl_count * 0.6):]

fall_mask = features_scaled['activity'] == 1
fall_data = features_scaled[fall_mask]
fall1 = fall_data.iloc[:int(fall_count * 0.6)]
fall2 = fall_data.iloc[int(fall_count * 0.6):]

print(adl1.shape, adl2.shape, fall1.shape, fall2.shape)

(142900, 10) (95267, 10) (127762, 10) (85176, 10)


In [19]:
def split_sequences(sequences, n_steps, stride):
    X, y = [], []
    for i in range(0, len(sequences) - n_steps + 1, stride):
        end_ix = i + n_steps
        
        seq_x = sequences.iloc[i:end_ix].iloc[:, :-1]  
        seq_y = sequences.iloc[end_ix-1].iloc[-1]      
        
        X.append(seq_x.values)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [20]:
WINDOW_LENGTH = 100
STRIDE = 25
train_data, train_labels = split_sequences(adl1, 
                                           WINDOW_LENGTH, 
                                          STRIDE)
print (f"after train(only adl) : {train_data.shape}")
print (train_labels.shape)

normal_test_data, normal_test_labels = split_sequences(adl2, 
                                                       WINDOW_LENGTH, 
                                                       STRIDE)
abnormal_test_data, abnormal_test_labels = split_sequences(fall2, 
                                                           WINDOW_LENGTH,
                                                           STRIDE)
print (f"after test adl(normal) : {normal_test_data.shape, normal_test_labels.shape}")
print (f"after test fall(abnormal) : {abnormal_test_data.shape, abnormal_test_labels.shape}")

after train(only adl) : (5713, 100, 9)
(5713,)
after test adl(normal) : ((3807, 100, 9), (3807,))
after test fall(abnormal) : ((3404, 100, 9), (3404,))


----

In [28]:
EPOCHS = 100
INITIAL_LEARNING_RATE = 1e-4  
DECAY_STEPS = 1000  
DECAY_RATE = 0.98  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

In [29]:
class SisFallDataset(Dataset):
    def __init__(self, data):
        # Transpose data from [batch, window_length, features] to [batch, features, window_length]
        data = data.transpose(0, 2, 1)
        self.data = torch.FloatTensor(data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]  # Returns shape [features, window_length]

In [30]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        middle_channels = out_channels // 2
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, middle_channels, kernel_size=2, padding='same'),
            nn.BatchNorm1d(middle_channels),
            nn.ReLU(),
            nn.Conv1d(middle_channels, out_channels, kernel_size=2, padding='same'),
            nn.BatchNorm1d(out_channels),   
            nn.ReLU()
        )
    def forward(self, x):
        return self.conv(x)

In [31]:
class SinusoidalPositionEmbeddings(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim

    def forward(self, time):
        device = time.device
        half_dim = self.dim // 2
        embeddings = math.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = time[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings

In [40]:
class CNNFeatureExtractor(nn.Module):
    def __init__(self, in_channels=9):
        super().__init__()
        self.inc = nn.Sequential(
            nn.Conv1d(in_channels, 64, kernel_size=3, padding='same'),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2)  
        )
        
        self.down = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, padding='same'),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.MaxPool1d(kernel_size=2) 
        )
        
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2), 
            nn.Conv1d(128, 64, kernel_size=3, padding='same'),
            nn.BatchNorm1d(64),
            nn.GELU()
        )

    def forward(self, x):
        x1 = self.inc(x)      # 64 channels
        x2 = self.down(x1)    # 128 channels
        x3 = self.up(x2)      # 64 channels
        return x2, (x1, x3)   # return: bottleneck(128), skip connections(64, 64)

In [41]:
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim=128, num_heads=8, num_layers=4):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim,
                nhead=num_heads,
                dim_feedforward=embed_dim*4,
                dropout=0.1,
                batch_first=True
            ) for _ in range(num_layers)
        ])
        
        self.pos_embedding = nn.Parameter(torch.zeros(1, 100//4, embed_dim))

    def forward(self, x):
        B, C, L = x.shape
        x = x.transpose(1, 2)
        x = x + self.pos_embedding
        
        for layer in self.layers:
            x = layer(x)
            
        return x.transpose(1, 2)

In [42]:
class TransUNet1D(nn.Module):
    def __init__(self, in_channels=9):
        super().__init__()
        self.cnn_encoder = CNNFeatureExtractor(in_channels)
        self.transformer = TransformerEncoder(embed_dim=128)
        
        self.time_embed = nn.Sequential(
            SinusoidalPositionEmbeddings(128),
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Linear(128, 128)
        )
        
        # Decoder
        self.up2 = nn.Sequential(
            nn.ConvTranspose1d(128, 64, kernel_size=2, stride=2),
            DoubleConv(128, 64)  # 128 because of skip connection (64+64)
        )
        
        self.up1 = nn.Sequential(
            nn.ConvTranspose1d(64, 32, kernel_size=2, stride=2),
            DoubleConv(64, 32)   # 64 because of skip connection (32+32)
        )
        
        self.final_conv = nn.Conv1d(32, in_channels, kernel_size=1)
        
    def forward(self, x, t):
        # Time embedding
        t_emb = self.time_embed(t)
        
        # CNN features
        features, skip_connections = self.cnn_encoder(x)  # features is 128 channels
        x1, x2 = skip_connections  # both are 64 channels
        
        # Add time information
        features = features + t_emb.unsqueeze(-1)
        
        # Transformer encoding
        features = self.transformer(features)
        
        # Decoder with skip connections
        x = self.up2[0](features)
        x = torch.cat([x, x2], dim=1)
        x = self.up2[1](x)
        
        x = self.up1[0](x)
        x = torch.cat([x, x1], dim=1)
        x = self.up1[1](x)
        
        return self.final_conv(x)

In [43]:
class DDPM:
    def __init__(self, num_timesteps=200): 
        self.num_timesteps = num_timesteps
        # Cosine schedule
        betas = torch.linspace(-6, 6, num_timesteps)
        betas = torch.sigmoid(betas) * (0.5e-2 - 1e-4) + 1e-4
        self.betas = betas
        self.alphas = 1. - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, dim=0)
        
    def q_sample(self, x_0, t):
        """
        Forward diffusion process: q(x_t | x_0)
        Samples from q(x_t | x_0) = N(x_t; √(αₜ)x₀, (1-αₜ)I)
        """
        sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod[t])[:, None, None]
        sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - self.alphas_cumprod[t])[:, None, None]
        epsilon = torch.randn_like(x_0)
        return sqrt_alphas_cumprod * x_0 + sqrt_one_minus_alphas_cumprod * epsilon, epsilon
    
    def sample_t(self, batch_size):
        """
        Uniformly sample timesteps t
        """
        return torch.randint(0, self.num_timesteps, (batch_size,))
        
    def p_sample(self, model, batch_size, device, shape):
        """
        Reverse diffusion process: p_θ(x_{t-1} | x_t)
        Generate samples using the reverse process
        """
        model.eval()
        with torch.no_grad():
            x_t = torch.randn(batch_size, *shape).to(device)
            
            for t in reversed(range(1, self.num_timesteps)):
                time_tensor = torch.ones(batch_size, dtype=torch.long).to(device) * t
                
                # Predict noise
                predicted_noise = model(x_t, time_tensor)
                
                alpha = self.alphas[t][:, None, None]
                alpha_cumprod = self.alphas_cumprod[t][:, None, None]
                beta = self.betas[t][:, None, None]
                
                # Add noise for stochasticity
                noise = torch.randn_like(x_t)
                
                # Compute mean for p_θ(x_{t-1} | x_t)
                x_t = 1 / torch.sqrt(alpha) * (
                    x_t - (1 - alpha) / torch.sqrt(1 - alpha_cumprod) * predicted_noise
                ) + torch.sqrt(beta) * noise
                
        model.train()
        return x_t

In [44]:
def train_ddpm(model, train_loader, ddpm, optimizer, scheduler, num_epochs, device):
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        n_batches = len(train_loader)
        
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            x_0 = batch.to(device)
            batch_size = x_0.shape[0]
            
            optimizer.zero_grad()
            t = ddpm.sample_t(batch_size).to(device)
            x_t, epsilon = ddpm.q_sample(x_0, t)
            epsilon_theta = model(x_t, t)
            loss = F.mse_loss(epsilon_theta, epsilon)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        scheduler.step()
        avg_loss = epoch_loss / n_batches
        print(f'Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_loss:.4f}')

In [45]:
model = TransUNet1D().to(device)
ddpm = DDPM(num_timesteps=100)  
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

print("Train data shape:", train_data.shape)

train_dataset = SisFallDataset(train_data)
sample_data = train_dataset[0]
print("Dataset sample shape:", sample_data.shape)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
first_batch = next(iter(train_loader))
print("Batch shape:", first_batch.shape)

train_ddpm(
    model=model,
    train_loader=train_loader,
    ddpm=ddpm,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=EPOCHS,
    device=device
)

Train data shape: (5713, 100, 9)
Dataset sample shape: torch.Size([9, 100])
Batch shape: torch.Size([32, 9, 100])


Epoch 1/100:   0%|                                      | 0/179 [00:00<?, ?it/s]


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 100 but got size 50 for tensor number 1 in the list.

------

In [ ]:
class SisFallDataset(Dataset):
    def __init__(self, data):
        # Transpose data from [batch, window_length, features] to [batch, features, window_length]
        data = data.transpose(0, 2, 1)
        self.data = torch.FloatTensor(data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]  # Returns shape [features, window_length]

In [ ]:
def detect_anomalies(model, ddpm, data_loader, device, threshold=None):
    model.eval()
    anomaly_scores = []

    with torch.no_grad():
        for batch in data_loader:
            x_0 = batch.to(device)
            batch_size = x_0.shape[0]

            t = ddpm.sample_t(batch_size).to(device)

            x_t, epsilon = ddpm.q_sample(x_0, t)

            epsilon_theta = model(x_t, t)

            # MSE Loss (Anomaly Score)
            loss = F.mse_loss(epsilon_theta, epsilon, reduction='none')
            loss = loss.mean(dim=(1, 2)) 
            anomaly_scores.extend(loss.cpu().numpy())  

    if threshold is None:
        threshold = np.percentile(anomaly_scores, 95)

    anomalies = [score > threshold for score in anomaly_scores]

    return anomalies, anomaly_scores, threshold


In [ ]:
normal_dataset = SisFallDataset(normal_test_data)
normal_loader = DataLoader(normal_dataset, batch_size=BATCH_SIZE)

abnormal_dataset = SisFallDataset(abnormal_test_data)
abnormal_loader = DataLoader(abnormal_dataset, batch_size=BATCH_SIZE)

In [ ]:
train_dataset = SisFallDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

train_anomalies, train_scores, _ = detect_anomalies(model, ddpm, train_loader, device)

plt.figure(figsize=(16,9))
plt.title('Loss Distribution of TRAIN ADL Data (NORMAL)', fontsize=16)
sns.histplot(train_scores, bins=20, kde=True, color='green')
plt.show()

In [ ]:
normal_anomalies, normal_scores, _ = detect_anomalies(model, ddpm, normal_loader, device)

plt.figure(figsize=(16,9))
plt.title('Loss Distribution of TEST ADL Data (NORMAL)', fontsize=16)
sns.histplot(normal_scores, bins=20, kde=True, color='blue')
plt.show()

In [ ]:
abnormal_anomalies, abnormal_scores, _ = detect_anomalies(model, ddpm, abnormal_loader, device)

plt.figure(figsize=(16,9))
plt.title('Loss Distribution of TEST FALL Data (ABNORMAL)', fontsize=16)
sns.histplot(abnormal_scores, bins=20, kde=True, color='red')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.title('MSE Distribution Comparison')
plt.hist(normal_scores, bins=50, alpha=0.5, color='blue', label='Normal')
plt.hist(abnormal_scores, bins=50, alpha=0.5, color='red', label='Abnormal')
plt.xlabel('Anomaly Score (MSE)')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
error_df = pd.DataFrame({
    'reconstruction_error': np.concatenate([normal_scores, abnormal_scores]),
    'true_class': np.concatenate([np.zeros(len(normal_scores)), np.ones(len(abnormal_scores))])
})

print ("error dataframe head : ")
print(error_df.head(10))
print ()
print ("error dataframe tail : ")
print(error_df.tail(10))
print ()
print(error_df.describe())
print ()
error_df['true_class'].value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='true_class', y='reconstruction_error', data=error_df, showfliers=False)
plt.ylabel('Reconstruction Error Distribution')
plt.xlabel('Class (0: Normal, 1: Abnormal)')
plt.show()

In [ ]:
threshold = np.percentile(normal_scores, 95)

In [ ]:
groups = error_df.groupby('true_class')
fig, ax = plt.subplots()

for name, group in groups:
    ax.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
            label="Normal" if name == 0 else "Fall")
ax.hlines(threshold, ax.get_xlim()[0], ax.get_xlim()[1], colors="r", zorder=100, label='Threshold')
ax.legend()
plt.title("Reconstruction Error by Class")
plt.ylabel("Reconstruction Error")
plt.xlabel("Data Point Index")
plt.show()

In [ ]:
LABELS = ["Normal", "Abnormal"]
y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
conf_matrix = confusion_matrix(error_df.true_class, y_pred)

plt.figure(figsize=(7, 7))
sns.heatmap(conf_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt="d")
plt.title("Confusion Matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, _ = precision_recall_fscore_support(error_df.true_class, y_pred, average='binary')
accuracy = accuracy_score(error_df.true_class, y_pred)

print('\nPerformance Metrics:')
print(f'Accuracy Score : {accuracy:.4f}')
print(f'Precision     : {precision:.4f}')
print(f'Recall        : {recall:.4f}')
print(f'F1 Score      : {f1:.4f}')